In [1]:
import numpy as np
from ezc3d import c3d
from pyomeca import Markers
import importlib
import os
import glob
from pathlib import Path
import dataprocesing as dp
import btk
import time

In [2]:
def read_labels(data_path,frame_rate):  
    c3d_to_compare= c3d(data_path)
    event = c3d_to_compare['parameters']['EVENT']['LABELS']['value']
    czas = np.around(c3d_to_compare['parameters']['EVENT']['TIMES']['value'][1]*frame_rate)
    eventy = [event, czas]
    
    eventy[0].index('Foot Strike')
    indxE = [i for i, x in enumerate(eventy[0]) if x == "Event"]
    indxFS = [i for i, x in enumerate(eventy[0]) if x == "Foot Strike"]

    CzasFS = np.zeros(len(indxFS))
    for i in range(len(indxFS)):
        CzasFS[i] = eventy[1][indxFS[i]]

    CzasE = np.zeros(len(indxE))
    for i in range(len(indxE)):
        CzasE[i] = eventy[1][indxE[i]]
    eventy[1].sort()

    p = []
    k = []
    for i in range(len(eventy[1])):
        if not i >= len(eventy[1])-2:
            pierwszy = eventy[1][i]
            drugi = eventy[1][i+1]
            trzeci = eventy[1][i+2]
            if pierwszy in CzasE:
                if drugi in CzasFS:
                    if trzeci in CzasE:
                        p.append(int(pierwszy))
                        k.append(int(trzeci))
    return [p,k]

In [3]:
def cropp_c3dfile(eventsFrame, filename, destiny):
    reader = btk.btkAcquisitionFileReader()
    reader.SetFilename(filename)
    reader.Update()
    acq = reader.GetOutput()
 
    writer = btk.btkAcquisitionFileWriter()
    
    for i in range(0, len(eventsFrame)-1):
        clone = acq.Clone();
        clone.ResizeFrameNumberFromEnd(acq.GetLastFrame() - eventsFrame[i][0] + 1)
        clone.ResizeFrameNumber(eventsFrame[i][1] - eventsFrame[i][0] + 1)
        clone.SetFirstFrame(eventsFrame[i][0])
        clone.ClearEvents()
        for e in btk.Iterate(acq.GetEvents()):
            if ((e.GetFrame() > clone.GetFirstFrame()) and (e.GetFrame() < clone.GetLastFrame())):
                  clone.AppendEvent(e)
        clone.SetFirstFrame(1)
        writer.SetInput(clone)
        if i > 8:
            writer.SetFilename(destiny + '\\' + (filename.split('\\')[-1]).split('.')[0]+ '-K' + str(i+1) + '.c3d')
            writer.Update()
        else:
            writer.SetFilename(destiny + '\\' + (filename.split('\\')[-1]).split('.')[0]+ '-K0' + str(i+1) + '.c3d')
            writer.Update()

In [4]:
def get_actor_name_from_file(filename):
    actor_name = filename.split('\\')[-1].split('-')[3]
    
    return actor_name

In [6]:
#     """
#     Funkcja tworzaca pliki, ktore zawieraja pojedynczy ruch, format pliku c3d
#     """
b = []
j = 0

destiny = r'Z:\testing'
for path in Path(r'Z:\Miecz').rglob('*.c3d'):
    if str(path).find('fail')==-1 and str(path).find('Cal')==-1:
        czas = read_labels(str(path),200)
        temp = []
        b.append(get_actor_name_from_file(str(path)))
        for i in range(len(czas[0])):
            if czas[0][i] != 0 and czas[1][i] != 0:
                temp.append([int(czas[0][i]), int(czas[1][i])-1])  
        if j > 1: 
            if b[0] == b[j-1]:
                cropp_c3dfile(temp, str(path), str(destiny))
            else:
                time.sleep(2)
                cropp_c3dfile(temp, str(path), str(destiny))
        j+=1
        print(path)

Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E01-T01.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E01-T02.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E01-T03.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E01-T04.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E01-T05.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E02-T01.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E02-T02.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E02-T03.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E02-T04.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E02-T05.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E03-T01.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E03-T02.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E03-T03.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S01\2020-08-05-B0444-S01-E03-T04.c3d
Z:\Miecz\B0444\2020-08-05-B0444-S0

Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E04-T01.c3d
Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E04-T02.c3d
Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E04-T03.c3d
Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E04-T04.c3d
Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E04-T05.c3d
Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E05-T01.c3d
Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E05-T02.c3d
Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E05-T03.c3d
Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E05-T04.c3d
Z:\Miecz\B0448\2020-08-07-B0448-S01\2020-08-07-B0448-S01-E05-T05.c3d
Z:\Miecz\B0449\2020-08-07-B0449-S01\2020-08-07-B0449-S01-E01-T01.c3d
Z:\Miecz\B0449\2020-08-07-B0449-S01\2020-08-07-B0449-S01-E01-T02.c3d
Z:\Miecz\B0449\2020-08-07-B0449-S01\2020-08-07-B0449-S01-E01-T03.c3d
Z:\Miecz\B0449\2020-08-07-B0449-S01\2020-08-07-B0449-S01-E01-T04.c3d
Z:\Miecz\B0449\2020-08-07-B0449-S0

Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E02-T03.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E02-T04.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E02-T05.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E03-T01.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E03-T02.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E03-T03.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E03-T04.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E03-T05.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E04-T01.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E04-T02.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E04-T03.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E04-T04.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E04-T05.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S01\2020-08-12-B0453-S01-E05-T01.c3d
Z:\Miecz\B0453\2020-08-12-B0453-S0

Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E01-T03.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E01-T04.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E01-T05.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E02-T01.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E02-T02.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E02-T03.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E02-T04.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E02-T05.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E03-T01.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E03-T02.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E03-T03.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E03-T04.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E03-T05.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S01\2020-08-17-B0458-S01-E04-T01.c3d
Z:\Miecz\B0458\2020-08-17-B0458-S0